# Model Training

The workshop contains three different notebooks. Each one focuses on a different stage:
    
1. Dataset Generation. The first notebook focuses on generating a dataset for training the model. We will create a Robust Test Suite to check that the dataset generated meets certain conditions
2. Model Training (This Notebook). The second notebook focuses on training the model. We will create a Robust Test Suite to check that the trained model meets certain conditions.
3. Model Inference. In the last notebook, we use mercury.monitoring to monitor data drift and estimate the predicted performance of the model without having the labels

## Setup

You can install mercury-robust by running:

```
!pip install mercury-robust
```

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score

SEED = 23

pd.set_option('display.max_colwidth', None)

## Read Dataset

Let's read the dataset that we generated in the first notebook

In [ ]:
path_dataset = "./dataset/"

df_train = pd.read_csv(path_dataset + "train.csv")
df_test = pd.read_csv(path_dataset + "test.csv")

In [ ]:
df_train.head()

## Train Model

Let's train a Decision Tree to predict if a client will default its credit

In [ ]:
label = "default.payment.next.month"
features = [c for c in df_train.columns if c!=label]

In [ ]:
X_train = df_train[features]
y_train = df_train[label]

X_test = df_test[features]
y_test = df_test[label]

model = DecisionTreeClassifier(random_state=SEED)
#model = DecisionTreeClassifier(
#    max_depth=6, class_weight="balanced", min_samples_split=15, min_samples_leaf=15, random_state=SEED
#)
model = model.fit(X_train, y_train)

## Evaluation

In [ ]:
acc_test = accuracy_score(y_test, model.predict(X_test))
auc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
f1_score_test = f1_score(y_test, model.predict(X_test))

print("accuracy: ", acc_test)
print("AUC: ", auc_test)
print("F1: ", f1_score_test)

## Robust Model Test Suite

As we did when creating the dataset, we will create a `TestSuite` using [mercury.robust](https://bbva.github.io/mercury-robust/). This time, we will focus on testing the trained model creating the next tests:

- [ModelSimplicityChecker](https://bbva.github.io/mercury-robust/reference/model_tests/#mercury.robust.model_tests.ModelSimplicityChecker): Looks if a trained model has a simple baseline which trained in the same dataset gives better or similar performance on a test dataset
- [CohortPerformanceTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.CohortPerformanceTest): looks if some metric performs poorly for some cohort of your data when compared with other groups
- [DriftMetricResistanceTest](https://bbva.github.io/mercury-robust/reference/model_tests/#mercury.robust.model_tests.DriftMetricResistanceTest): Checks the robustness of a trained model to drift in the inputs of the data.
- [TreeCoverageTest](https://bbva.github.io/mercury-robust/reference/model_tests/#mercury.robust.model_tests.TreeCoverageTest): Checks whether a given test_dataset covers a minimum of all the branches of a tree

In [ ]:
# Load Data Schema
from mercury.dataschema import DataSchema
schema = DataSchema.load(path_dataset + "schema.json")

In [ ]:
from mercury.robust.model_tests import (
    ModelSimplicityChecker,
    DriftMetricResistanceTest,
    TreeCoverageTest
)
from mercury.robust.data_tests import CohortPerformanceTest
from mercury.robust import TestSuite

def create_model_test_suite(
    model, 
    X_train, 
    y_train,
    X_test,
    y_test,
    schema,
    add_tree_coverage_test=False
):
    
    model_tests = []
    
    # Model Simpclicity Checker
    model_simplicity_checker = ModelSimplicityChecker(
        model = model,
        X_train = X_train,
        y_train = y_train,
        X_test = X_test,
        y_test = y_test,
        threshold = 0.02,
        eval_fn = roc_auc_score,
        ignore_feats=label,
        dataset_schema=schema,
        baseline_model=LogisticRegression(solver='liblinear', class_weight='balanced')
    )
    model_tests.append(model_simplicity_checker)
    
    # Cohort Performance Test
    group = "SEX"
    def eval_precision(df):
        return precision_score(df[label], df["prediction"])

    # Calculate predictions, we will use this in one test
    df_test_pred = pd.DataFrame()
    df_test_pred[group] = X_test[group].values
    df_test_pred["prediction"] = model.predict(X_test)
    df_test_pred[label] = y_test
    cohort_perf_test = CohortPerformanceTest(
        name="precision_by_gender_check",
        base_dataset=df_test_pred, group_col="SEX", eval_fn = eval_precision, threshold = 0.05,
        threshold_is_percentage=False
    )
    model_tests.append(cohort_perf_test)
    
    # One DriftMetricResistanceTest for each variable
    for f in features:
        drift_args = None
        if ('BILL_AMT' in f) or ('PAY_AMT' in f):
            drift_args = {'cols': [f], 'force': df_train[f].quantile(q=0.25)}
        elif 'PAY_' in f:
            drift_args = {'cols': [f], 'force': 2}
        if drift_args is not None:
            model_tests.append(DriftMetricResistanceTest(
                model = model,
                X = X_test,
                Y = y_test,
                drift_type = 'shift_drift',
                drift_args = drift_args,
                tolerance = 0.05,
                eval=accuracy_score,
                name="drift resistance " + f
            ))
        
    # Tree Coverage Test(only if specified)
    if add_tree_coverage_test:
        tree_coverage_test = TreeCoverageTest(model, X_test, threshold_coverage=.75)
        model_tests.append(tree_coverage_test)
    
    # Create Suite
    test_suite = TestSuite(
        tests=model_tests
    )
    
    return test_suite

In [ ]:
test_suite = create_model_test_suite(
    model, 
    X_train, 
    y_train,
    X_test,
    y_test,
    schema,
    add_tree_coverage_test=True
)
test_results = test_suite.run()

In [ ]:
test_suite.get_results_as_df()

## Save Model

In [ ]:
path_model = "./models/"

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
from joblib import dump
dump(model, path_model + 'model.joblib') 

In [ ]:
import pickle

with open(path_model + "features.pkl", "wb") as fp:
    pickle.dump(features, fp)